# Mateo de la Cuadra Copetta
Github:         mateodlcc  
n° de alumno:   21637407   

<h1 style="text-align: center;">
[IIC2613] - Inteligencia Artificial
</h1>
<h2 style="text-align: center;"> Tarea 5: Localización de Rostros </h2>

Para el desarrollo de la tarea se utilizará YOLO V8.  

Para ello usaremos las siguientes librerías:

| Librerías     | Instalación |
|-----------    |:-----------:|
|ultralytics    |pip3 install ultralytics|
| torch         |pip uninstall torch && pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118|
| PIL           |pip install pillow|
|os             | - |
|opencv (cv2)   | pip install opencv-python|
|mss            | pip install mss|

In [2]:
import pandas as pd
import random
import shutil
import os
from ultralytics import YOLO
from mss import mss
import numpy as np
import cv2
import torch

# Preprocesamiento de datos

In [2]:
# Leer el dataset
df = pd.read_csv('faces.csv')

# Crear la carpeta labels
if not os.path.exists('labels'):
    os.makedirs('labels')

# Crear los archivos .txt
for image in df['image_name'].unique():
    with open(f'labels/{image.replace(".jpg", ".txt")}', 'w') as file:
        for _, row in df[df['image_name'] == image].iterrows():
            x_center = (row['x0'] + row['x1']) / (2 * row['width'])
            y_center = (row['y0'] + row['y1']) / (2 * row['height'])
            width = (row['x1'] - row['x0']) / row['width']
            height = (row['y1'] - row['y0']) / row['height']
            file.write(f'0 {x_center} {y_center} {width} {height}\n')

# Obtener la lista de imagenes y mezclarla
images = df['image_name'].unique()
random.shuffle(images)

# Separar la lista en train, test y validation (70%, 20%, 10%)
train_images = images[:int(0.7 * len(images))]
test_images = images[int(0.7 * len(images)):int(0.9 * len(images))]
validation_images = images[int(0.9 * len(images)):]

# Crear las carpetas
# train
if not os.path.exists('train'):
    os.makedirs('train')
if not os.path.exists('train/images'):
    os.makedirs('train/images')
if not os.path.exists('train/labels'):
    os.makedirs('train/labels')

# test
if not os.path.exists('test'):
    os.makedirs('test')
if not os.path.exists('test/images'):
    os.makedirs('test/images')
if not os.path.exists('test/labels'):
    os.makedirs('test/labels')

# validation
if not os.path.exists('validation'):
    os.makedirs('validation')
if not os.path.exists('validation/images'):
    os.makedirs('validation/images')
if not os.path.exists('validation/labels'):
    os.makedirs('validation/labels')

# Copiar las imagenes a las carpetas
for image in train_images:
    shutil.copy(f'images/{image}', 'train/images')
for image in test_images:
    shutil.copy(f'images/{image}', 'test/images')
for image in validation_images:
    shutil.copy(f'images/{image}', 'validation/images')

# Copiar los labels a las carpetasn
for image in train_images:
    shutil.copy(f'labels/{image.replace(".jpg", ".txt")}', 'train/labels')
for image in test_images:
    shutil.copy(f'labels/{image.replace(".jpg", ".txt")}', 'test/labels')
for image in validation_images:
    shutil.copy(f'labels/{image.replace(".jpg", ".txt")}', 'validation/labels')

# Mostrar el resultado
print('Dataset transformado exitosamente')
print(f'Train: {len(train_images)}')
print(f'Test: {len(test_images)}')
print(f'Validation: {len(validation_images)}')

Dataset transformado exitosamente
Train: 1542
Test: 441
Validation: 221


# Entrenamiento

In [7]:
if torch.cuda.is_available():
    print("CUDA está disponible. Los modelos usarán la GPU.")
    device = torch.device("cuda")
else:
    print("CUDA no está disponible. Los modelos usarán la CPU.")
    device = torch.device("cpu")

print(device)

CUDA está disponible. Los modelos usarán la GPU.
cuda


A continuación tenemos los modelos disponibles de Yolo V8, nosotros usaremos YOLOv8n:
| Model | Size (pixels) | mAPval50-95 | SpeedCPU ONNX(ms) | SpeedA100 TensorRT(ms) | Params(M) | FLOPs(B) |
|-------|---------------|-------------|--------------------|------------------------|-----------|----------|
| YOLOv8n | 640 | 37.3 | 80.4 | 0.99 | 3.2 | 8.7 |
| YOLOv8s | 640 | 44.9 | 128.4 | 1.20 | 11.2 | 28.6 |
| YOLOv8m | 640 | 50.2 | 234.7 | 1.83 | 25.9 | 78.9 |
| YOLOv8l | 640 | 52.9 | 375.2 | 2.39 | 43.7 | 165.2 |
| YOLOv8x | 640 | 53.9 | 479.1 | 3.53 | 68.2 | 257.8 |

In [8]:
# Cargamos el modelo
model = YOLO("yolov8n.pt")
model = model.to(device)

Establecemos el número de épocas que usaremos para el entrenamiento y el path actual (para establecer un path común)

In [9]:
epochs = 100
batch = 16
PATH = os.path.join(os.getcwd(), "data")

In [10]:
# Entrenamos el modelo
data_path = os.path.join(PATH, "data.yaml")
results = model.train(data=data_path, epochs=epochs, batch=batch)

New https://pypi.org/project/ultralytics/8.2.53 available  Update with 'pip install -U ultralytics'
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=d:\Trabajos_UC\9 noveno semestre\IA\mateo\data\data.yaml, epochs=100, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cuda:0, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, box

# Test del modelo

Se usara el modelo entrenado 'best.pt' para evaluar el modelo caprutando la pantalla en tiempo real

In [3]:
device = torch.device("cpu")

# Cargamos el modelo
model_path = '/results/weights/best.pt'
model = YOLO("best2.pt")
model = model.to(device)

# Inicializamos la captura de pantalla
sct = mss()
bounding_box = {"top": 0, "left": 0, "width": 800, "height": 600}

while True:
    sct_img = sct.grab(bounding_box)
    frame = np.array(sct_img)
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Realizamos la deteccion
    results = model(frame)

    # Dibujamos los bounding boxes
    for result in results[0].boxes:  # Accede a los resultados de las cajas
        x1, y1, x2, y2 = result.xyxy[0].cpu().numpy().astype(int)
        conf = result.conf[0].cpu().numpy()
        cls = int(result.cls[0].cpu().numpy())
        
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Dibujar el rectángulo
        label = f'{model.names[cls]} {conf:.2f}'  # Crear la etiqueta
        cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)  # Añadir la etiqueta


    # Mostramos el resultado
    cv2.imshow("frame", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        break

WARNING  best2.pt appears to require 'dill', which is not in ultralytics requirements.
AutoInstall will run now for 'dill' but this feature will be removed in the future.
Recommend fixes are to train a new model using the latest 'ultralytics' package or to run a command with an official YOLOv8 model, i.e. 'yolo predict model=yolov8n.pt'


requirements: Ultralytics requirement ['dill'] not found, attempting AutoUpdate...
   ---------------------------------------- 116.3/116.3 kB 6.6 MB/s eta 0:00:00

requirements: AutoUpdate success  8.1s, installed 1 package: ['dill']
requirements:  Restart runtime or rerun command for updates to take effect


0: 480x640 (no detections), 621.2ms
Speed: 20.2ms preprocess, 621.2ms inference, 13.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 525.7ms
Speed: 9.8ms preprocess, 525.7ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 420.5ms
Speed: 10.6ms preprocess, 420.5ms inference, 2.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 445.7ms
Speed: 6.6ms preprocess, 445.7ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 648.4ms
Speed: 9.7ms preprocess, 648.4ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 

: 

In [3]:
device = torch.device("cpu")

# Cargamos el modelo
model_path = '/results/weights/best.pt'
model = YOLO("best2.pt")
model = model.to(device)


# Ahora con la camara
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()

    # Realizamos la deteccion
    results = model(frame)

    # Dibujamos los bounding boxes
    for result in results[0].boxes:  # Accede a los resultados de las cajas
        x1, y1, x2, y2 = result.xyxy[0].cpu().numpy().astype(int)
        conf = result.conf[0].cpu().numpy()
        cls = int(result.cls[0].cpu().numpy())
        
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Dibujar el rectángulo
        label = f'{model.names[cls]} {conf:.2f}'  # Crear la etiqueta
        cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)  # Añadir la etiqueta

    # Mostramos el resultado
    cv2.imshow("frame", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        cap.release()
        break



0: 480x640 (no detections), 487.6ms
Speed: 12.6ms preprocess, 487.6ms inference, 16.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Face, 345.9ms
Speed: 8.0ms preprocess, 345.9ms inference, 7.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Face, 300.4ms
Speed: 8.0ms preprocess, 300.4ms inference, 6.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Face, 328.3ms
Speed: 8.4ms preprocess, 328.3ms inference, 9.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Face, 420.3ms
Speed: 0.0ms preprocess, 420.3ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Face, 400.9ms
Speed: 8.0ms preprocess, 400.9ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 Faces, 339.1ms
Speed: 11.1ms preprocess, 339.1ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Face, 278.2ms
Speed: 0.0ms preprocess, 278.2ms inference, 0.0ms postprocess per image at sh

# Test del modelo

Se usara el modelo entrenado 'best.pt' para evaluar el modelo caprutando la pantalla en tiempo real

# Cara mas cercana

Finalmente, nos interesa quedarnos con la cara mas cercana, el criterio para determinar aquello será el Bounding Box mas grande.

In [3]:
device = torch.device("cpu")

# Cargamos el modelo
model_path = '/results/weights/best.pt'
model = YOLO("best2.pt")
model = model.to(device)

# Bucasmos el BB más grande dentro de la imagen, primero probaremos con la imagen de la cámara
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()

    # Realizamos la deteccion
    results = model(frame)

    # Encontramos el bounding box mas grande
    max_area = 0
    max_bb = None
    for result in results[0].boxes:  # Accede a los resultados de las cajas
        x1, y1, x2, y2 = result.xyxy[0].cpu().numpy().astype(int)
        area = (x2 - x1) * (y2 - y1)
        if area > max_area:
            max_area = area
            max_bb = result

    if max_bb is not None:
        x1, y1, x2, y2 = max_bb.xyxy[0].cpu().numpy().astype(int)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        label = f'{model.names[int(max_bb.cls[0].cpu().numpy())]} {max_bb.conf[0].cpu().numpy():.2f}'
        cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        print(f'Centro: {(x1 + x2) / 2}, {(y1 + y2) / 2}')

    # Mostramos el resultado
    cv2.imshow("frame", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        cap.release()
        break



0: 480x640 3 Faces, 279.7ms
Speed: 8.0ms preprocess, 279.7ms inference, 14.0ms postprocess per image at shape (1, 3, 480, 640)



Centro: 512.0, 191.5


0: 480x640 1 Face, 227.7ms
Speed: 6.0ms preprocess, 227.7ms inference, 8.0ms postprocess per image at shape (1, 3, 480, 640)



Centro: 344.0, 172.5


0: 480x640 1 Face, 232.1ms
Speed: 4.0ms preprocess, 232.1ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)



...

Centro: 167.5, 146.5


0: 480x640 3 Faces, 215.6ms
Speed: 3.0ms preprocess, 215.6ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)



Centro: 187.5, 145.0


0: 480x640 4 Faces, 214.9ms
Speed: 4.1ms preprocess, 214.9ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)


Centro: 219.0, 145.5
